<a href="https://colab.research.google.com/github/Ludvins/Practicas_PDGE/blob/master/CUDA/Multiplicacion_matrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Luis Antonio Ortega Andrés    
Antonio Coín Castro*

# Ejercicio opcional: multiplicación de matrices

Consideramos el problema de multiplicar dos matrices cuadradas $A$ y $B$ de dimensiones $N\times N$, obteniendo como resultado una matriz $C$, también $N\times N$.

## Multiplicación en CPU

Realizamos la multiplicación en CPU, cuya eficiencia es $O(N^3)$. 

In [46]:
%%writefile matmul_cpu.cu

#include <stdio.h>

#define N 64

void matrixMultCPU(float a[N][N], float b[N][N], float c[N][N]) {
  int i, j, k;
  for (i=0; i < N; i++) {
    for (j = 0; j < N; j++) {
      float sum = 0;
      for (k = 0; k < N; k++) {
        sum += a[i][k] * b[k][j];
      }
      c[i][j] = sum;
    }
  }
}

int main() {
  float a[N][N], b[N][N], c[N][N];

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  /* inicializando variables con datos*/
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      a[i][j] = 1.0;
      b[i][j] = 2.0;
    }
  }

  int nIter=1000;
  cudaEventRecord(start);
  for (int i = 0; i < nIter; i++)
    matrixMultCPU(a, b, c);

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  float msecTotal = 0.0;
  cudaEventElapsedTime(&msecTotal, start, stop);

  // Comprueba resultados
  int errores = 0;
  for (int y = 0; y < N; y++) {
    for (int x = 0; x < N; x++) {
        if (c[y][x] != 2*N) {
          errores++;
        }
    }
  }

  printf("Producto de matrices %d x %d\n", N, N);
  printf("Resultado ");
  if (errores == 0){
    printf("correcto\n");
  }
  else {
    printf("incorrecto. Errores: %d\n", errores);
    return 1;
  }

  float msecPerKernelExecution = msecTotal / nIter;
  double flopsPerMMull = 2.0 * N * N * N;
  double gigaFlops = (flopsPerMMull * 1.0e-9) /
    (msecPerKernelExecution / 1000.0);

  printf("Tiempo medio de ejecución: %f ms\n", msecPerKernelExecution);
  printf("GFLOPS: %f\n", gigaFlops);
  return 0;
}

Overwriting matmul_cpu.cu


In [47]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_cpu.cu -o matmul_cpu -lcudadevrt
!./matmul_cpu

Producto de matrices 64 x 64
Resultado correcto
Tiempo medio de ejecución: 0.748456 ms
GFLOPS: 0.700493


## Multiplicación en GPU

Pasamos ahora a realizar la misma implementación pero en GPU. La adaptamos para un tamaño $N$ arbitrario, empleando siempre bloques de $8\times 8$ threads (para aprovechar al máximo el paralelismo, como ya comentamos en el ejercicio sobre la suma de matrices), y un número de bloques igual a $(N+7)/8$. Diferenciamos según si se utiliza memoria compartida o no.

### Sin usar memoria compartida

Implementamos en primer lugar el algoritmo más sencillo. Establecemos una estructura bidimensional de bloques y threads, y el thread con identificador $(i, j)$ (en coordenadas de grid) procesa la fila $j$-ésima y la columna $i$-ésima para obtener el elemento $C_{ij}$. De esta forma obtenemos del orden de $N^2$ threads, donde cada uno realiza $O(N)$ operaciones.

In [48]:
%%writefile matmul_gpu_v1.cu

#include <stdio.h>

#define N 64
#define TPB 8

// Computa la multiplicación de matrices en GPU sin memoria compartida
__global__ void matrixMultGPU(int n, float *a, float *b, float *c) {
  int k;
  float sum = 0;
  int col = threadIdx.x + blockDim.x * blockIdx.x;
  int row = threadIdx.y + blockDim.y * blockIdx.y;
  
  if (row < n && col < n) {
    for (k = 0; k < n; k++) {
      sum += a[row * n + k] * b[k * n + col];
    }
    c[row * n + col] = sum;
  }
}

int main() {
  float a[N][N], b[N][N], c[N][N];
  float *dev_a, *dev_b, *dev_c;
  int size = N * N * sizeof(float);

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaMalloc((void **) &dev_a, size);
  cudaMalloc((void **) &dev_b, size);
  cudaMalloc((void **) &dev_c, size);

  // inicializando variables
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      a[i][j] = 1.0;
      b[i][j] = 2.0;
    }
  }

  cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);

  // Tamaño de grid y bloque
  dim3 dimGrid((N+TPB-1)/TPB, (N+TPB-1)/TPB);
  dim3 dimBlock(TPB, TPB);

  int nIter=1000;
  cudaEventRecord(start);
  for (int i=0; i <nIter; i++) {
    matrixMultGPU<<<dimGrid, dimBlock>>>(N, dev_a, dev_b, dev_c);
  }
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);

  float msecTotal = 0.0;
  cudaEventElapsedTime(&msecTotal, start, stop);

  // Comprueba resultados
  int errores = 0;
  for (int y = 0; y < N; y++) {
    for (int x = 0; x < N; x++) {
        if (c[y][x] != 2*N) {
          errores++;
          //printf("[%d][%d]=%f en vez de %d; ", y, x, c[y][x], 2*N);
        }
    }
    //printf("\n");
  }

  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);

  printf("Producto de matrices %d x %d\n", N, N);
  printf("Resultado ");
  if (errores == 0){
    printf("correcto\n");
  }
  else {
    printf("incorrecto. Errores: %d\n", errores);
    return 1;
  }

  float msecPerKernelExecution = msecTotal / nIter;
  double flopsPerMMull = 2.0 * N * N * N;
  double gigaFlops = (flopsPerMMull * 1.0e-9) /
    (msecPerKernelExecution / 1000.0);

  printf("GFLOPS: %f\n", gigaFlops);

  return 0;
}

Overwriting matmul_gpu_v1.cu


In [50]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v1.cu -o matmul_gpu_v1 -lcudadevrt
!nvprof ./matmul_gpu_v1

==18441== NVPROF is profiling process 18441, command: ./matmul_gpu_v1
Producto de matrices 64 x 64
Resultado correcto
GFLOPS: 57.092081
==18441== Profiling application: ./matmul_gpu_v1
==18441== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.89%  7.9416ms      1000  7.9410us  7.8710us  13.152us  matrixMultGPU(int, float*, float*, float*)
                    0.07%  5.5360us         2  2.7680us  2.6240us  2.9120us  [CUDA memcpy HtoD]
                    0.04%  2.9120us         1  2.9120us  2.9120us  2.9120us  [CUDA memcpy DtoH]
      API calls:   94.77%  177.07ms         2  88.535ms     865ns  177.07ms  cudaEventCreate
                    2.90%  5.4198ms      1000  5.4190us  4.2400us  45.584us  cudaLaunchKernel
                    1.87%  3.4890ms         1  3.4890ms  3.4890ms  3.4890ms  cudaEventSynchronize
                    0.18%  339.38us         1  339.38us  339.38us  339.38us  cuDeviceTotalMem
              

Vemos que se realiza adecuadamente el producto y no se producen errores. Además, el tiempo medio por ejecución es de tan solo 8 us, muy por debajo del tiempo medio de ejecución en CPU.

## Memoria compartida con optimizaciones adicionales

Pasamos ahora a optimizar la forma de cálculo del producto. En concreto hacemos tres optimizaciones:

- Utilizamos memoria compartida para el acceso a los datos. De esta forma, definiremos "mosaicos" para hacer el producto de matrices por bloques, de manera independiente y optimizando el acceso a memoria. Así conseguimos que las matrices $A$ y $B$ se carguen solo $N/8$ veces en memoria, al contrario que antes que se cargaban $N$ veces. Necesitamos usar `__syncthreads()` para asegurarnos de que todos los datos necesarios han sido cargados.
- Accedemos a los mosaicos de forma traspuesta para aumentar la eficiencia en memoria, debido a que se guardan en "column-major".
- Desenrrollamos bucles para optimización del compilador. Utilizamos la directiva del precompilador `#pragma unroll`.

También añadimos una mejora, y es que, además de permitir tamaños arbitrariamente grandes de $N$ (limitados solo por el tamaño máximo de grid), **permitimos que no tenga que ser potencia de 2**. En este casod debemos asegurarnos que los mosaicos que solo estén parcialmente rellenos tengan a 0 el resto de sus posiciones, para no influir en el cálculo. Como siempre, debemos comprobar que los índices en el acceso a matrices sean correctos.

In [67]:
%%writefile matmul_gpu_v2.cu

#include <stdio.h>

#define N 64
#define TPB 8

__global__ void matrixMultGPU2(int n, float* A, float* B, float* C) {
    float sum = 0;
    int tile;
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int i = blockIdx.x * blockDim.x + tx;
    int j = blockIdx.y * blockDim.y + ty;

      // Mosaicos en memoria de bloque
      __shared__ float As[TPB][TPB];
      __shared__ float Bs[TPB][TPB];

      // Recorre los mosaicos de A y B necesarios para computar la submatriz de C
      for (tile = 0; tile < (n+TPB-1)/TPB; tile++){
          // Carga los mosaicos de A y B en paralelo (y de forma traspuesta)
          if ((ty + (tile*TPB))<n && i < n)
            As[ty][tx] = A[(i * n) + (ty + (tile*TPB))];
          else
            As[ty][tx] = 0.0;
          if ((tx + (tile*TPB))<n && j < n)
            Bs[ty][tx] = B[((tx + (tile * TPB))*n) + j];
          else
            Bs[ty][tx] = 0.0;

            __syncthreads();

            // Computa los resultados para la submatriz de C (también traspuestos)
    #pragma unroll
            for (int k = 0; k < TPB; k++)
              sum += As[k][tx] * Bs[ty][k];

            __syncthreads();
      }
      // Escribe en paralelo los resultados obtenidos por el bloque
      if (i < n && j < n)
        C[i * n + j] = sum;
}

int main() {
  float a[N][N], b[N][N], c[N][N];
  float *dev_a, *dev_b, *dev_c;
  int size = N * N * sizeof(float);

  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);

  cudaMalloc((void **) &dev_a, size);
  cudaMalloc((void **) &dev_b, size);
  cudaMalloc((void **) &dev_c, size);

  // inicializando variables
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++) {
      a[i][j] = 1.0;
      b[i][j] = 2.0;
    }
  }

  cudaMemcpy(dev_a, a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(dev_b, b, size, cudaMemcpyHostToDevice);

  // Tamaño de grid y bloque
  dim3 dimGrid((N+TPB-1)/TPB, (N+TPB-1)/TPB);
  dim3 dimBlock(TPB, TPB);

  int nIter=1000;
  cudaEventRecord(start);
  for (int i=0; i <nIter; i++) {
    matrixMultGPU2<<<dimGrid, dimBlock>>>(N, dev_a, dev_b, dev_c);
  }
  cudaEventRecord(stop);
  cudaEventSynchronize(stop);

  cudaMemcpy(c, dev_c, size, cudaMemcpyDeviceToHost);

  float msecTotal = 0.0;
  cudaEventElapsedTime(&msecTotal, start, stop);

  // Comprueba resultados
  int errores = 0;
  for (int y = 0; y < N; y++) {
    for (int x = 0; x < N; x++) {
        if (c[y][x] != 2*N) {
          errores++;
          //printf("[%d][%d]=%f en vez de %d; ", y, x, c[y][x], 2*N);
        }
    }
    //printf("\n");
  }

  cudaFree(dev_a);
  cudaFree(dev_b);
  cudaFree(dev_c);

  printf("Producto de matrices %d x %d\n", N, N);
  printf("Resultado ");
  if (errores == 0){
    printf("correcto\n");
  }
  else {
    printf("incorrecto. Errores: %d\n", errores);
    return 1;
  }

  float msecPerKernelExecution = msecTotal / nIter;
  double flopsPerMMull = 2.0 * N * N * N;
  double gigaFlops = (flopsPerMMull * 1.0e-9) /
    (msecPerKernelExecution / 1000.0);

  printf("GFLOPS: %f\n", gigaFlops);

  return 0;
}

Overwriting matmul_gpu_v2.cu


In [68]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v2.cu -o matmul_gpu_v2 -lcudadevrt
!nvprof ./matmul_gpu_v2

==18956== NVPROF is profiling process 18956, command: ./matmul_gpu_v2
Producto de matrices 64 x 64
Resultado correcto
GFLOPS: 54.544425
==18956== Profiling application: ./matmul_gpu_v2
==18956== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.90%  8.3038ms      1000  8.3030us  8.2550us  9.4710us  matrixMultGPU2(int, float*, float*, float*)
                    0.07%  5.5040us         2  2.7520us  2.5920us  2.9120us  [CUDA memcpy HtoD]
                    0.03%  2.8800us         1  2.8800us  2.8800us  2.8800us  [CUDA memcpy DtoH]
      API calls:   94.87%  187.79ms         2  93.894ms     948ns  187.79ms  cudaEventCreate
                    3.13%  6.1919ms      1000  6.1910us  4.9960us  43.777us  cudaLaunchKernel
                    1.54%  3.0434ms         1  3.0434ms  3.0434ms  3.0434ms  cudaEventSynchronize
                    0.19%  378.22us         1  378.22us  378.22us  378.22us  cuDeviceTotalMem
             

Podemos probar a incrementar el tamaño de las matrices, modificarlo para que no sean potencia de 2, e incluso variar el número de hilos por bloque. Comprobamos que en todo caso el resultado obtenido con esta versión sigue siendo correcto.

In [55]:
!sed -i '/#define N/c\#define N 200' matmul_gpu_v2.cu
!sed -i '/#define TPB/c\#define TPB 32' matmul_gpu_v2.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v2.cu -o matmul_gpu_v2 -lcudadevrt
!nvprof ./matmul_gpu_v2

==18649== NVPROF is profiling process 18649, command: ./matmul_gpu_v2
Producto de matrices 200 x 200
Resultado correcto
GFLOPS: 101.464006
==18649== Profiling application: ./matmul_gpu_v2
==18649== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.97%  156.43ms      1000  156.43us  154.14us  158.33us  matrixMultGPU2(int, float*, float*, float*)
                    0.02%  32.992us         2  16.496us  15.744us  17.248us  [CUDA memcpy HtoD]
                    0.01%  14.399us         1  14.399us  14.399us  14.399us  [CUDA memcpy DtoH]
      API calls:   52.49%  175.09ms         2  87.545ms     882ns  175.09ms  cudaEventCreate
                   45.56%  151.97ms         1  151.97ms  151.97ms  151.97ms  cudaEventSynchronize
                    1.62%  5.4186ms      1000  5.4180us  4.3190us  26.699us  cudaLaunchKernel
                    0.11%  355.71us         1  355.71us  355.71us  355.71us  cuDeviceTotalMem
          

## Medición de tiempos

Medimos los tiempos medios de ejecución y los GFLOPS con/sin optimizaciones, usando tamaños de bloque $TPB=8$ y $TPB=32$.

### Sin memoria compartida

In [43]:
!sed -i '/#define TPB/c\#define TPB 8' matmul_gpu_v1.cu
!for i in 16 32 64 128 512; do sed -i "/#define N/c\#define N $i" matmul_gpu_v1.cu && /usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v1.cu -o matmul_gpu_v1 -lcudadevrt && nvprof ./matmul_gpu_v1; done

==17834== NVPROF is profiling process 17834, command: ./matmul_gpu_v1
Producto de matrices 16 x 16
Resultado correcto
GFLOPS: 1.424414
==17834== Profiling application: ./matmul_gpu_v1
==17834== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.82%  2.9292ms      1000  2.9290us  2.8800us  4.0960us  matrixMultGPU(int, float*, float*, float*)
                    0.11%  3.2960us         2  1.6480us  1.4080us  1.8880us  [CUDA memcpy HtoD]
                    0.06%  1.8880us         1  1.8880us  1.8880us  1.8880us  [CUDA memcpy DtoH]
      API calls:   96.61%  184.26ms         2  92.128ms     970ns  184.26ms  cudaEventCreate
                    2.85%  5.4376ms      1000  5.4370us  4.3540us  26.589us  cudaLaunchKernel
                    0.21%  394.92us         1  394.92us  394.92us  394.92us  cuDeviceTotalMem
                    0.12%  228.63us         3  76.211us  4.6430us  216.77us  cudaMalloc
                    0.10%

In [44]:
!sed -i '/#define TPB/c\#define TPB 32' matmul_gpu_v1.cu
!for i in 16 32 64 128 512; do sed -i "/#define N/c\#define N $i" matmul_gpu_v1.cu && /usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v1.cu -o matmul_gpu_v1 -lcudadevrt && nvprof ./matmul_gpu_v1; done

==18083== NVPROF is profiling process 18083, command: ./matmul_gpu_v1
Producto de matrices 16 x 16
Resultado correcto
GFLOPS: 1.387842
==18083== Profiling application: ./matmul_gpu_v1
==18083== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.88%  4.3709ms      1000  4.3700us  4.2560us  5.4080us  matrixMultGPU(int, float*, float*, float*)
                    0.07%  3.2640us         2  1.6320us  1.4080us  1.8560us  [CUDA memcpy HtoD]
                    0.04%  1.8880us         1  1.8880us  1.8880us  1.8880us  [CUDA memcpy DtoH]
      API calls:   96.53%  179.14ms         2  89.568ms     905ns  179.13ms  cudaEventCreate
                    3.01%  5.5852ms      1000  5.5850us  4.3710us  48.141us  cudaLaunchKernel
                    0.19%  346.47us         1  346.47us  346.47us  346.47us  cuDeviceTotalMem
                    0.08%  147.08us        97  1.5160us     137ns  58.408us  cuDeviceGetAttribute
               

### Con memoria compartida y optimizaciones




In [36]:
!sed -i '/#define TPB/c\#define TPB 8' matmul_gpu_v2.cu
!for i in 16 32 64 128 512; do sed -i "/#define N/c\#define N $i" matmul_gpu_v2.cu && /usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v2.cu -o matmul_gpu_v2 -lcudadevrt && nvprof ./matmul_gpu_v2; done

==16756== NVPROF is profiling process 16756, command: ./matmul_gpu_v2
Producto de matrices 16 x 16
Resultado correcto
GFLOPS: 1.184703
==16756== Profiling application: ./matmul_gpu_v2
==16756== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.85%  3.5306ms      1000  3.5300us  3.4870us  4.7040us  matrixMultGPU2(int, float*, float*, float*)
                    0.09%  3.2630us         2  1.6310us  1.4080us  1.8550us  [CUDA memcpy HtoD]
                    0.05%  1.8880us         1  1.8880us  1.8880us  1.8880us  [CUDA memcpy DtoH]
      API calls:   96.20%  185.52ms         2  92.758ms     883ns  185.52ms  cudaEventCreate
                    3.36%  6.4740ms      1000  6.4730us  5.0840us  28.879us  cudaLaunchKernel
                    0.18%  346.39us         1  346.39us  346.39us  346.39us  cuDeviceTotalMem
                    0.07%  143.39us         3  47.797us  2.9930us  135.64us  cudaMalloc
                    0.07

In [39]:
!sed -i '/#define TPB/c\#define TPB 32' matmul_gpu_v2.cu
!for i in 16 32 64 128 512; do sed -i "/#define N/c\#define N $i" matmul_gpu_v2.cu && /usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v2.cu -o matmul_gpu_v2 -lcudadevrt && nvprof ./matmul_gpu_v2; done

==17507== NVPROF is profiling process 17507, command: ./matmul_gpu_v2
Producto de matrices 16 x 16
Resultado correcto
GFLOPS: 0.683744
==17507== Profiling application: ./matmul_gpu_v2
==17507== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.95%  10.752ms      1000  10.752us  10.687us  11.328us  matrixMultGPU2(int, float*, float*, float*)
                    0.03%  3.2630us         2  1.6310us  1.4070us  1.8560us  [CUDA memcpy HtoD]
                    0.02%  1.8560us         1  1.8560us  1.8560us  1.8560us  [CUDA memcpy DtoH]
      API calls:   93.51%  180.41ms         2  90.204ms     853ns  180.41ms  cudaEventCreate
                    3.16%  6.0890ms         1  6.0890ms  6.0890ms  6.0890ms  cudaEventSynchronize
                    2.88%  5.5527ms      1000  5.5520us  4.4640us  26.735us  cudaLaunchKernel
                    0.20%  390.81us         1  390.81us  390.81us  390.81us  cuDeviceTotalMem
              

### Otro tipo de dato

Usamos matrices de tipo `double` en vez de `float`.

In [78]:
!sed 's/float/double/g' matmul_gpu_v1.cu > matmul_gpu_v1_double.cu
!sed -i '/double msecTotal = 0.0/c\float msecTotal = 0.0;' matmul_gpu_v1_double.cu
!sed 's/float/double/g' matmul_gpu_v2.cu > matmul_gpu_v2_double.cu
!sed -i '/double msecTotal = 0.0/c\float msecTotal = 0.0;' matmul_gpu_v2_double.cu

Medimos los tiempos en este caso, con y sin memoria compartida, fijando $TPB=32$.

In [79]:
!sed -i '/#define TPB/c\#define TPB 32' matmul_gpu_v1_double.cu
!for i in 16 32 64 128 512; do sed -i "/#define N/c\#define N $i" matmul_gpu_v1_double.cu && /usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v1_double.cu -o matmul_gpu_v1_double -lcudadevrt && nvprof ./matmul_gpu_v1_double; done

==19425== NVPROF is profiling process 19425, command: ./matmul_gpu_v1_double
Producto de matrices 16 x 16
Resultado correcto
GFLOPS: 0.692034
==19425== Profiling application: ./matmul_gpu_v1_double
==19425== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.95%  10.580ms      1000  10.579us  10.463us  12.736us  matrixMultGPU(int, double*, double*, double*)
                    0.03%  3.5190us         2  1.7590us  1.4720us  2.0470us  [CUDA memcpy HtoD]
                    0.02%  1.9200us         1  1.9200us  1.9200us  1.9200us  [CUDA memcpy DtoH]
      API calls:   93.74%  184.42ms         2  92.211ms     955ns  184.42ms  cudaEventCreate
                    3.13%  6.1590ms      1000  6.1590us  4.9660us  30.352us  cudaLaunchKernel
                    2.68%  5.2783ms         1  5.2783ms  5.2783ms  5.2783ms  cudaEventSynchronize
                    0.18%  347.13us         1  347.13us  347.13us  347.13us  cuDeviceTotalMe

In [80]:
!sed -i '/#define TPB/c\#define TPB 32' matmul_gpu_v2_double.cu
!for i in 16 32 64 128 512; do sed -i "/#define N/c\#define N $i" matmul_gpu_v2_double.cu && /usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true matmul_gpu_v2_double.cu -o matmul_gpu_v2_double -lcudadevrt && nvprof ./matmul_gpu_v2_double; done

==19678== NVPROF is profiling process 19678, command: ./matmul_gpu_v2_double
Producto de matrices 16 x 16
Resultado correcto
GFLOPS: 0.239859
==19678== Profiling application: ./matmul_gpu_v2_double
==19678== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.98%  32.914ms      1000  32.914us  32.703us  33.471us  matrixMultGPU2(int, double*, double*, double*)
                    0.01%  3.4870us         2  1.7430us  1.4720us  2.0150us  [CUDA memcpy HtoD]
                    0.01%  1.9200us         1  1.9200us  1.9200us  1.9200us  [CUDA memcpy DtoH]
      API calls:   86.05%  215.35ms         2  107.67ms     939ns  215.35ms  cudaEventCreate
                   11.25%  28.154ms         1  28.154ms  28.154ms  28.154ms  cudaEventSynchronize
                    2.27%  5.6895ms      1000  5.6890us  4.4040us  52.180us  cudaLaunchKernel
                    0.22%  556.37us         1  556.37us  556.37us  556.37us  cuDeviceTotalM

### Comparación con el programa de ejemplo de CUDA

Finalmente, comparamos los resultados con el programa de ejemplo *built-in* que proporciona CUDA para hacer el producto de matrices.

In [122]:
%cd /usr/local/cuda/samples/0_Simple/matrixMul
!make

/usr/local/cuda-10.1/samples/0_Simple/matrixMul
/usr/local/cuda-10.1/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o matrixMul matrixMul.o 
mkdir -p ../../bin/x86_64/linux/release
cp matrixMul ../../bin/x86_64/linux/release


In [119]:
!nvprof ./matrixMul

[Matrix Multiply Using CUDA] - Starting...
==20719== NVPROF is profiling process 20719, command: ./matrixMul
GPU Device 0: "Tesla T4" with compute capability 7.5

MatrixA(320,320), MatrixB(640,320)
Computing result using CUDA Kernel...
done
Performance= 406.26 GFlop/s, Time= 0.323 msec, Size= 131072000 Ops, WorkgroupSize= 1024 threads/block
Checking computed result for correctness: Result = PASS

NOTE: The CUDA Samples are not meant for performancemeasurements. Results may vary when GPU Boost is enabled.
==20719== Profiling application: ./matrixMul
==20719== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.82%  96.735ms       301  321.38us  318.68us  323.16us  void MatrixMulCUDA<int=32>(float*, float*, float*, int, int)
                    0.12%  111.68us         2  55.839us  39.552us  72.127us  [CUDA memcpy HtoD]
                    0.07%  66.463us         1  66.463us  66.463us  66.463us  [CUDA memcpy DtoH]
     

Vemos que utiliza bloques de $32\times 32$ threads, y que tiene una eficiencia de unos 400 GFLOPS para tamaños de matriz en torno a 300, lo cual supera a todos nuestros programas.